### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Post-Traumatic_Stress_Disorder/GSE860'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = None  # set to different values when applicable
age_row = None  # set to different values when applicable
gender_row = None  # set to different values when applicable

# Sample Characteristics Dictionary parsing based on example provided
sample_chars = {
    2: "Post-Traumatic_Stress_Disorder: PTSD; Control",
    3: "age: 35",
    4: "gender: Male"
}

# Check available keys for Post-Traumatic_Stress_Disorder, age, and gender
for key, value in sample_chars.items():
    if 'Post-Traumatic_Stress_Disorder' in value:
        trait_row = key
    elif 'age' in value:
        age_row = key
    elif 'gender' in value:
        gender_row = key

def convert_trait(value):
    if not value or ':' not in value:
        return None
    trait = value.split(':')[1].strip().lower()
    return 1 if trait == 'ptsd' else 0 if trait == 'control' else None

def convert_age(value):
    if not value or ':' not in value:
        return None
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    if not value or ':' not in value:
        return None
    gender = value.split(':')[1].strip().lower()
    return 1 if gender == 'male' else 0 if gender == 'female' else None

save_cohort_info('GSE860', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    clinical_data = geo_select_clinical_features(clinical_data, 'Post-Traumatic_Stress_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/trait_data/GSE860.csv'
    clinical_data.to_csv(csv_path)
    print(preview_df(clinical_data))
else:
    print("No trait data available.")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Ensure `requires_gene_mapping` is defined


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 6: Gene Identifier Mapping

In [ ]:
if requires_gene_mapping:
    # 1. Identify the keys for identifiers and gene symbols
    identifier_key = 'Name'
    gene_symbol_key = 'Desc'

    # Print columns in gene_annotation for debugging
    print("Columns in gene_annotation:", gene_annotation.columns)
    
    # 2. Get the dataframe storing the mapping between probe IDs and genes
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # Rename the columns to match expected format regardless of initial naming
    gene_mapping.rename(columns={identifier_key: 'ID', gene_symbol_key: 'Gene'}, inplace=True)
    
    # Print columns in gene_mapping after renaming for debugging
    print("Columns in gene_mapping after renaming:", gene_mapping.columns)
    
    # 3. Apply the mapping and get the resulting gene expression dataframe
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    
    # Optionally, print the first few rows of the gene_data to verify
    print("Mapped Gene Data preview:")
    print(preview_df(gene_data))


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/gene_data/GSE860.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Post-Traumatic_Stress_Disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE860', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/GSE860.csv'
    unbiased_merged_data.to_csv(csv_path)
